<a href="https://colab.research.google.com/github/youngsikwon/Py_ML_Deep_Learning/blob/master/Apple.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from keras.models import Sequential
from keras.layers import LSTM, Dropout, Dense, Activation

import datetime



In [4]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving AAPL.csv to AAPL.csv
User uploaded file "AAPL.csv" with length 93886 bytes


In [5]:
# Load Dataset

data = pd.read_csv('AAPL.csv')
data.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2015-01-16,107.029999,107.580002,105.199997,105.989998,97.385445,78513300
1,2015-01-20,107.839996,108.970001,106.500000,108.720001,99.893822,49899900
2,2015-01-21,108.949997,111.059998,108.269997,109.550003,100.656441,48575900
3,2015-01-22,110.260002,112.470001,109.720001,112.400002,103.275063,53796400
4,2015-01-23,112.300003,113.750000,111.529999,112.980003,103.807991,46464800


In [0]:
#Compute Mid Price

high_prices = data['High'].values
low_prices = data['Low'].values

mid_prices = (high_prices + low_prices) / 2

In [0]:
# Create WIndows

seq_len = 50
sequence_length = seq_len + 1

result = []

for index in range(len(mid_prices)- sequence_length):
  result.append(mid_prices[index : index + sequence_length])

In [12]:
normalized_data = []
for window in result:
    normalized_window = [((float(p) / float(window[0])) - 1) for p in window]
    normalized_data.append(normalized_window)

result = np.array(normalized_data)

# split train and test data
row = int(round(result.shape[0] * 0.9))
train = result[:row, :]
np.random.shuffle(train)

x_train = train[:, :-1]
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))
y_train = train[:, -1]

x_test = result[row:, :-1]
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))
y_test = result[row:, -1]

x_train.shape, x_test.shape

((1086, 50, 1), (121, 50, 1))

In [22]:
#Build a Model

model = Sequential()

model.add(LSTM(50, return_sequences=True, input_shape=(50, 1)))

model.add(LSTM(64, return_sequences=False))

model.add(Dense(1, activation='linear'))

model.compile(loss='mse', optimizer='rmsprop')

model.summary()


Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_14 (LSTM)               (None, 50, 50)            10400     
_________________________________________________________________
lstm_15 (LSTM)               (None, 64)                29440     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 65        
Total params: 39,905
Trainable params: 39,905
Non-trainable params: 0
_________________________________________________________________


In [24]:
model.fit(x_train, y_train,
    validation_data=(x_test, y_test),
    batch_size=10,
    epochs=20)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Train on 1086 samples, validate on 121 samples
Epoch 1/20





1086/1086 [==============================] - 10s 9ms/step - loss: 0.0018 - val_loss: 0.0012
Epoch 2/20
1086/1086 [==============================] - 7s 7ms/step - loss: 7.3837e-04 - val_loss: 6.7509e-04
Epoch 3/20
1086/1086 [==============================] - 8s 7ms/step - loss: 4.9385e-04 - val_loss: 4.5421e-04
Epoch 4/20
1086/1086 [==============================] - 8s 7ms/step - loss: 4.1301e-04 - val_loss: 3.6990e-04
Epoch 5/20
1086/1086 [==============================] - 8s 7ms/step - loss: 3.5798e-04 - val_loss: 7.4490e-04
Epoch 6/20
1086/1086 [==============================] - 8s 7ms/step - loss: 2.9753e-04 - val_loss: 5.0347e-04
Epoch 7/20
1086/1086 [==============================] - 8s 7ms/step - loss: 2.6841e-04 - val_loss: 3.0003e-04
Epoch 8/20
1086/1086 [==============================] - 8s 7ms/step - loss: 2.5239e-04 -